<a href="https://colab.research.google.com/github/soutrik7771/CARDIO-VASCULAR-DISORDER-PREDICTION/blob/master/social_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.set_option("display.max_rows",None)


In [0]:
data=pd.read_csv("train.csv")

In [0]:
data.head()

,timestamp,main_meter,sub_meter_1,sub_meter_2,building_number
0,2017-04-01 00:00:00,2898.969633,1390.244624,0.000000,1
1,2017-04-01 00:15:00,2531.896894,1087.313897,0.000000,1
2,2017-04-01 00:30:00,2171.649138,695.625737,0.484790,1
3,2017-04-01 00:45:00,2207.299159,615.496571,1.317875,1
4,2017-04-01 01:00:00,2839.441067,1071.573744,0.000000,1


In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132000 entries, 0 to 131999
Data columns (total 5 columns):
timestamp          132000 non-null object
main_meter         132000 non-null float64
sub_meter_1        132000 non-null float64
sub_meter_2        132000 non-null float64
building_number    132000 non-null int64
dtypes: float64(3), int64(1), object(1)
memory usage: 5.0+ MB


In [0]:
data.shape

(132000, 5)

In [0]:
data.isnull().sum()

timestamp          0
main_meter         0
sub_meter_1        0
sub_meter_2        0
building_number    0
dtype: int64

In [0]:
data["building_number"].unique()

array([1, 2, 3, 4, 5])

In [0]:
data.describe()

,main_meter,sub_meter_1,sub_meter_2,building_number
count,132000.000000,132000.000000,132000.000000,132000.000000
mean,6095.767107,2092.888851,776.830989,3.000000
std,3485.125319,1135.885749,883.495350,1.414219
min,61.635635,0.000000,0.000000,1.000000
25%,3455.911410,1365.005373,4.857540,2.000000
50%,5362.188090,2075.676367,621.910511,3.000000
75%,7685.405459,2772.505817,1365.898059,4.000000
max,23174.609258,17206.162311,4392.073840,5.000000


In [0]:
# we will split the dataframe into multiple such datframes

In [0]:
df_lst=[pd.DataFrame(y) for x, y in data.groupby("building_number", as_index=False)]

In [0]:
len(df_lst)

5

In [0]:
df1=pd.DataFrame(df_lst[0]).drop("building_number",axis=1)
df2=pd.DataFrame(df_lst[1]).drop("building_number",axis=1)
df3=pd.DataFrame(df_lst[2]).drop("building_number",axis=1)
df4=pd.DataFrame(df_lst[3]).drop("building_number",axis=1)
df5=pd.DataFrame(df_lst[4]).drop("building_number",axis=1)

In [0]:
df1.head()

,timestamp,main_meter,sub_meter_1,sub_meter_2
0,2017-04-01 00:00:00,2898.969633,1390.244624,0.000000
1,2017-04-01 00:15:00,2531.896894,1087.313897,0.000000
2,2017-04-01 00:30:00,2171.649138,695.625737,0.484790
3,2017-04-01 00:45:00,2207.299159,615.496571,1.317875
4,2017-04-01 01:00:00,2839.441067,1071.573744,0.000000


In [0]:
lst=[df1,df2,df3,df4,df5]
for i in range(0,len(lst)):
  lst[i]["timestamp"]=pd.to_datetime(lst[i]["timestamp"] , format = '%Y/%m/%d %H:%M:%S')
  lst[i].set_index(["timestamp"],inplace=True)

In [0]:
df1.head()

,main_meter,sub_meter_1,sub_meter_2
timestamp,,,
2017-04-01 00:00:00,2898.969633,1390.244624,0.000000
2017-04-01 00:15:00,2531.896894,1087.313897,0.000000
2017-04-01 00:30:00,2171.649138,695.625737,0.484790
2017-04-01 00:45:00,2207.299159,615.496571,1.317875
2017-04-01 01:00:00,2839.441067,1071.573744,0.000000


In [0]:
#creating the train and validation set
train = df1[:int(0.8*(len(df1)))]
valid = df1[int(0.8*(len(df1))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/vector_ar/var_model.py:461: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  params = np.linalg.lstsq(z, y_sample)[0]


In [0]:
type(prediction)

numpy.ndarray

In [0]:
prediction

array([[2527.42422139, 1173.02487327,   12.96510147],
       [2591.66443007, 1204.31393939,   25.3074229 ],
       [2652.6659853 , 1233.42257471,   37.06785964],
       ...,
       [4209.39350181, 1833.99750175,  318.32111356],
       [4209.39350181, 1833.99750175,  318.32111356],
       [4209.39350181, 1833.99750175,  318.32111356]])

In [0]:
cols=df1.columns.tolist()

In [0]:
cols

['main_meter', 'sub_meter_1', 'sub_meter_2']

In [0]:
#converting predictions to dataframe
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols]) # creating blank
# filling each row of the dataframe
for j in range(0,len(cols)):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]


In [0]:
pred.head()

,main_meter,sub_meter_1,sub_meter_2
0,2527.42,1173.02,12.9651
1,2591.66,1204.31,25.3074
2,2652.67,1233.42,37.0679
3,2710.69,1260.57,48.2836
4,2765.96,1285.93,58.9885


In [0]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [0]:
test=pd.read_csv("test.csv")

In [0]:
df_test_lst=[pd.DataFrame(y) for x, y in test.groupby("building_number", as_index=False)]

In [0]:
df1_test=pd.DataFrame(df_test_lst[0]).drop("building_number",axis=1)
df2_test=pd.DataFrame(df_test_lst[1]).drop("building_number",axis=1)
df3_test=pd.DataFrame(df_test_lst[2]).drop("building_number",axis=1)
df4_test=pd.DataFrame(df_test_lst[3]).drop("building_number",axis=1)
df5_test=pd.DataFrame(df_test_lst[4]).drop("building_number",axis=1)

In [0]:
lst1=[df1_test,df2_test,df3_test,df4_test,df5_test]
for i in range(0,len(lst1)):
  lst1[i]["timestamp"]=pd.to_datetime(lst1[i]["timestamp"] , format = '%Y/%m/%d %H:%M:%S')
  lst1[i].set_index(["timestamp"],inplace=True)

In [0]:
df1_test.head()

""
timestamp
2018-01-01 00:00:00
2018-01-01 00:15:00
2018-01-01 00:30:00
2018-01-01 00:45:00
2018-01-01 01:00:00


In [0]:
train=[df1,df2,df3,df4,df5]
testt=[df1_test,df2_test,df3_test,df4_test,df5_test]
pred2=pd.DataFrame()
for i in range(len(train)):
  trn=train[i].copy()
  tst=testt[i].copy()
  #model and predict
  model = VAR(endog=trn)
  model_fit = model.fit()
  prediction = model_fit.forecast(model_fit.y, steps=len(tst))
  #converting predictions to dataframe
  pred = pd.DataFrame(index=range(0,len(prediction)),columns=cols) # creating blank
  # filling each row of the dataframe
  for j in range(0,len(cols)):
    for i in range(0, len(prediction)):
      pred.iloc[i][j] = prediction[i][j]

  pred2=pd.concat([pred2,pred],ignore_index=True)

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/vector_ar/var_model.py:461: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  params = np.linalg.lstsq(z, y_sample)[0]


In [0]:
test.shape

(51617, 2)

In [0]:
pred2.shape

(51617, 3)

In [0]:
pred3=pd.concat([test,pred2],axis=1)
pred3.columns

Index(['timestamp', 'building_number', 'main_meter', 'sub_meter_1',
       'sub_meter_2'],
      dtype='object')

In [108]:
pred3.head()

,timestamp,main_meter,sub_meter_1,sub_meter_2,building_number
0,2018-01-01 00:00:00,2270.16,902.056,18.6437,1
1,2018-01-01 00:15:00,2370.49,946.038,31.797,1
2,2018-01-01 00:30:00,2465.01,987.16,44.2988,1
3,2018-01-01 00:45:00,2554.13,1025.63,56.187,1
4,2018-01-01 01:00:00,2638.22,1061.64,67.4972,1


In [0]:
pred3=pred3[["timestamp","main_meter","sub_meter_1","sub_meter_2","building_number"]]

In [113]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [0]:
pred3.to_csv('data.csv')
!cp data.csv drive/My\ Drive/